In [12]:
import gplearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [13]:
# import dataset (cleaned)
df = pd.read_csv(r'cleaned_data_simplified.csv', index_col=0)

In [1]:
df.columns

NameError: name 'df' is not defined

In [15]:
x = df.drop('Duration', axis=1)

In [16]:
y = df.loc[:,'Duration'].to_frame()
type(y)
print(y)

         Duration
0               3
1              24
2               8
3               8
4               4
...           ...
9830302        39
9830304        70
9830305        52
9830306        67
9830307        58

[8769371 rows x 1 columns]


In [17]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=10)

In [18]:
from gplearn.genetic import SymbolicRegressor
import pickle

In [19]:
gp = SymbolicRegressor(population_size=100, stopping_criteria=0.01, verbose=1, random_state=10,
                       tournament_size=10, p_crossover=0.7,p_hoist_mutation=0.00,
                       p_point_mutation=0.15,p_subtree_mutation=0.15, n_jobs=5, init_method='grow',
                       function_set=('add','sub','mul','div','sqrt'))

In [32]:
# gp.set_params(warm_start = True, generations=50, n_jobs=3)

SymbolicRegressor(function_set=('add', 'sub', 'mul', 'div', 'sqrt'),
                  generations=50, init_method='grow', n_jobs=3, p_crossover=0.7,
                  p_hoist_mutation=0.0, p_point_mutation=0.15,
                  p_subtree_mutation=0.15, population_size=100, random_state=10,
                  stopping_criteria=0.01, tournament_size=10, verbose=1,
                  warm_start=True)

In [20]:
# TODO train once some system resources have been freed up
gp.fit(X=X_train, y=np.ravel(y_train))

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     4.16           6796.9        3          14.4839              N/A      3.10m
   1     2.99          87.6349        6          13.7195              N/A      2.02m
   2     3.13          104.212        6          13.7195              N/A      1.85m
   3     4.24          85.1298        6          13.4113              N/A      1.78m
   4     5.05          1369.43       12          13.1934              N/A      1.82m
   5     6.89          514.055       10          12.7985              N/A      1.87m
   6     7.42          56.9199       10          12.7985              N/A      1.76m
   7     9.34          47.8789       10          12.7985              N/A      1.86m
   8     9.47          543.759       10          12.4866              N/A  

SymbolicRegressor(function_set=('add', 'sub', 'mul', 'div', 'sqrt'),
                  init_method='grow', n_jobs=5, p_crossover=0.7,
                  p_hoist_mutation=0.0, p_point_mutation=0.15,
                  p_subtree_mutation=0.15, population_size=100, random_state=10,
                  stopping_criteria=0.01, tournament_size=10, verbose=1)